In [1]:
path = "D:\\Rythm\\"

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import FastICA, PCA
import scipy as sc
from scipy.stats import kurtosis
from scipy.stats import skew
import pywt
from scipy import stats
from statsmodels.robust import stand_mad

# Functions

In [17]:
# Quality of hypnogram recording : if there is much than p percent of the record that is -1 -> bad recording

def hypnogram_quality(hypx , treshold):
    return 0 if hypx.count(-1)/len(hypx) > treshold else 1

In [18]:
def preprocessing_hypno(col):
    col = col.replace("'","").replace(",","")[1:-1].split(' ')
    res = [int(i) for i in col]
    return res

In [19]:
# Count number of breaks of slow wave sleep

def count_breaks_3(hypx):
    count = 0
    for i in range(0,len(hypx)-1):
        if hypx[i] !=3 & hypx[i+1] == 3:
            count = count + 1
    return count
            

In [20]:
def get_hypno(data):
    return data["HYPNOGRAM"]

In [21]:
def get_eeg(data):
    return data.ix[:,2:-1]

In [22]:
def get_eeg_features(EEG, age):
    age["mean_eeg"] = EEG.apply(np.mean,axis = 1)
    age["std_eeg"] = EEG.apply(np.std , axis = 1)
    age["kurt_eeg"] =  EEG.apply(sc.stats.kurtosis , axis =1)
    age["skew_eeg"] = EEG.apply(sc.stats.skew , axis =1)
    age["max_eeg"] = EEG.apply(np.max , axis = 1)
    age["min_eeg"] = EEG.apply(np.min, axis = 1)
    return age

In [36]:
def get_hypno_features(age,hyp):
    #age["mean_hypno"] = hyp.apply(np.mean)
    #age["std_hypno"] = hyp.apply(np.std)
    #age["kurt_hypno"] = hyp.apply(sc.stats.kurtosis)
    #age["skew_hypno"] = hyp.apply(sc.stats.skew)

    age["length_list"] = hyp.apply(lambda hypx : len(hypx))
    #age["occurencies_-1"] = hyp.apply(lambda hypx : hypx.count(-1))
    age["occurencies_0"] = hyp.apply(lambda hypx : hypx.count(0))
    age["occurencies_1"] = hyp.apply(lambda hypx : hypx.count(1))
    age["occurencies_2"] = hyp.apply(lambda hypx : hypx.count(2))
    age["occurencies_3"] = hyp.apply(lambda hypx : hypx.count(3))
    age["occurencies_4"] = hyp.apply(lambda hypx : hypx.count(4))

    # Quality of recording : categorical variable
    quality_threshold = 0.1 #10% of well recorded time steps
    age["record_quality"] = hyp.apply(lambda hypx : hypnogram_quality(hypx,quality_threshold))

    # Percentage of Slow Wave Sleep (3)

    age["SlowWaveSleep_percentage"] = age["occurencies_3"] / age["length_list"]

    # Number of breaks in Slow Wave Sleep

    age["breaks"] = hyp.apply(count_breaks_3)
    return age

In [109]:
def get_device(age,data):
    age["device"] = data["DEVICE"]
    return age

In [25]:
def get_fft_length(EEG , age):
    length = []
    for j in range(0,len(EEG)):
        fft = np.fft.fft(EEG.ix[j,:])
        freqs = np.fft.fftfreq(len(fft) , d = 1/250) # frequence
        y = np.sqrt(np.square(fft.real) + np.square(fft.imag)) # amplitude
        z = y[(0 < freqs) & (freqs < 4)]
        length.append(len(z))
    age["fft_length"] = length
    return age
        

In [26]:
def get_fft_mean_ampl(EEG , age):
    res = []
    for j in range(0,len(EEG)):
        fft = np.fft.fft(EEG.ix[j,:])
        freqs = np.fft.fftfreq(len(fft) , d = 1/250) # frequence
        y = np.sqrt(np.square(fft.real) + np.square(fft.imag)) # amplitude
        z = y[(0 < freqs) & (freqs < 4)]
        res.append(np.mean(z))
    age["fft_amp"] = res
    return age

In [13]:
def get_wavelet(EEG):
    coef = EEG.apply(lambda x : pywt.wavedec(x, 'db8', level=8, mode='per'), axis =1)
    return coef

In [33]:
def get_wav_features(coef, age):
    max_coef = []
    min_coef = []
    mean_coef = []
    median_coef = []
    for j in range(0,len(coef)):
        temp_max = [np.max(coef[j][i]) for i in range(0,8)]
        max_coef.append(np.max(temp_max))
        temp_min = [np.min(coef[j][i]) for i in range(0,8)]
        min_coef.append(np.min(temp_min))
        temp_mean = [np.mean(coef[j][i]) for i in range(0,8)]
        mean_coef.append(np.mean(temp_mean))
        temp_median = [np.median(coef[j][i]) for i in range(0,8)]
        median_coef.append(np.median(temp_median))
    age["wav_max"] = max_coef
    age["wav_min"] = min_coef
    age["wav_mean"] = mean_coef
    age["wav_median"] = median_coef
    return age

In [28]:
def get_first_level(coef , age_wav):
  
    wav = []
    for i in range(0,len(coef[1][0])):
        wav.append("wav_"+str(i))

    bla = []
    for i in range(0,581):
        bla.append(coef[i][0])
    
    res = pd.DataFrame(bla)
    res.columns = wav
    
    return pd.concat([age_wav , res] , axis =1)

In [91]:
def wav_features(age_wav, coef):
    
    n = np.shape(age_wav)[0]
    for j in range(0,9):

        age_wav["wav_min"+str(j)] = pd.Series()
        age_wav["wav_max"+str(j)] = pd.Series()
        age_wav["wav_mean"+str(j)] = pd.Series()
        age_wav["wav_std"+str(j)] = pd.Series()
    
    for i in range(0,n):
        for j in range(0,9):
        
            age_wav.ix[i,"wav_min"+str(j)]= np.min(coef[i][j])
            age_wav.ix[i,"wav_max"+str(j)] = np.max(coef[i][j])
            age_wav.ix[i,"wav_mean"+str(j)] = np.mean(coef[i][j])
            age_wav.ix[i,"wav_std"+str(j)] = np.std(coef[i][j])
    return age_wav

# Load data

In [27]:
dat2 = pd.read_csv(path+"train_input.csv" , header = 0 , sep =";")

In [74]:
age = pd.read_table(path+"agetrain.txt" , header = 0 , sep =";")
age_wav = pd.DataFrame.copy(age)
age_wav_feat = pd.DataFrame.copy(age)

# Features from EEG

In [66]:
EEG = get_eeg(dat2)
age = get_device(age,dat2)
age = get_eeg_features(EEG,age)

# Features from HYPNOGRAM

In [67]:
hypnogram = get_hypno(dat2)
hyp = hypnogram.apply(preprocessing_hypno)

In [68]:
age = get_hypno_features(age,hyp)
age = get_fft_length(EEG, age)
age = get_fft_mean_ampl(EEG,age)

In [69]:
age.head()

,ID,TARGET,device,mean_eeg,std_eeg,kurt_eeg,skew_eeg,max_eeg,min_eeg,length_list,occurencies_0,occurencies_1,occurencies_2,occurencies_3,occurencies_4,record_quality,SlowWaveSleep_percentage,breaks,fft_length,fft_amp
0,0,32,0,1.395848e-03,26.811350,129.538478,-2.978869,370.873413,-584.737732,971,89,7,443,233,177,1,0.239959,94,1199,30110.757853
1,1,29,0,-6.471835e-04,11.881911,0.421025,-0.042748,58.905529,-66.786278,789,78,19,379,198,115,1,0.250951,50,1199,12311.275042
2,2,36,0,1.041976e-02,97.740294,215.544923,3.324026,1944.065430,-2119.885986,295,53,0,53,188,0,1,0.637288,31,1199,122016.340524
3,3,56,0,-7.526106e-03,17.135283,145.579363,4.544951,440.595032,-190.482651,896,65,5,302,189,323,1,0.210938,56,1199,19151.960999
4,4,60,1,-5.648294e-08,0.000034,1.346067,-0.322313,0.000124,-0.000185,939,463,58,143,239,36,1,0.254526,33,1199,0.038199


In [54]:
age.to_csv("feat.csv")

# Wavelet decomposition

In [70]:
age_wav = get_device(age_wav , dat2)

In [71]:
coef = get_wavelet(EEG)

In [72]:
age_wav = wav_features(age_wav,coef)

In [73]:
age_wav.head()

,ID,TARGET,device,wav_min0,wav_max0,wav_mean0,wav_std0,wav_min1,wav_max1,wav_mean1,...,wav_mean6,wav_std6,wav_min7,wav_max7,wav_mean7,wav_std7,wav_min8,wav_max8,wav_mean8,wav_std8
0,0,32,0,-1402.798386,739.813544,4.700932e-02,129.593857,-2681.838399,1995.227763,3.070288,...,1.520851e-02,4.628216,-35.175841,33.788319,-4.650481e-05,1.364214,-11.422063,4.929972,-2.596736e-04,0.156237
1,1,29,0,-239.334369,245.921723,-3.019311e-02,90.242721,-202.026861,239.183099,-3.649749,...,-4.621617e-02,6.917315,-6.526320,7.033817,-5.799839e-05,1.560359,-0.768676,0.833141,1.334174e-05,0.150531
2,2,36,0,-2674.792171,3918.960693,1.302607e-01,400.859404,-6443.665322,4284.004186,6.807693,...,1.440450e-02,8.770716,-24.799147,19.927536,1.321510e-04,1.605641,-8.060059,4.324459,-1.771710e-04,0.159557
3,3,56,0,-595.274283,720.434080,-1.184204e-01,90.606810,-780.744970,1499.878949,5.064747,...,7.282955e-04,1.466351,-2.671790,3.093451,-1.654635e-05,0.279525,-5.199568,2.359752,-1.168486e-04,0.041260
4,4,60,1,-0.000517,0.000626,-8.460627e-07,0.000196,-0.001050,0.001245,-0.000015,...,4.602178e-08,0.000005,-0.000023,0.000023,-5.696861e-10,0.000002,-0.000024,0.000026,6.542273e-09,0.000001


In [53]:
age_wav.to_csv("wav.csv")

# Train a random forest

In [75]:
EEG = get_eeg(dat2)
age = get_device(age,dat2)
age = get_eeg_features(EEG,age)



In [76]:
EEG = get_eeg(dat2)
age = get_device(age,dat2)
age = get_eeg_features(EEG,age)

hypnogram = get_hypno(dat2)
hyp = hypnogram.apply(preprocessing_hypno)

age = get_hypno_features(age,hyp)

In [77]:
age.head()

,ID,TARGET,device,mean_eeg,std_eeg,kurt_eeg,skew_eeg,max_eeg,min_eeg,length_list,occurencies_0,occurencies_1,occurencies_2,occurencies_3,occurencies_4,record_quality,SlowWaveSleep_percentage,breaks
0,0,32,0,1.395848e-03,26.811350,129.538478,-2.978869,370.873413,-584.737732,971,89,7,443,233,177,1,0.239959,94
1,1,29,0,-6.471835e-04,11.881911,0.421025,-0.042748,58.905529,-66.786278,789,78,19,379,198,115,1,0.250951,50
2,2,36,0,1.041976e-02,97.740294,215.544923,3.324026,1944.065430,-2119.885986,295,53,0,53,188,0,1,0.637288,31
3,3,56,0,-7.526106e-03,17.135283,145.579363,4.544951,440.595032,-190.482651,896,65,5,302,189,323,1,0.210938,56
4,4,60,1,-5.648294e-08,0.000034,1.346067,-0.322313,0.000124,-0.000185,939,463,58,143,239,36,1,0.254526,33


In [77]:
EEG.head()

,EEG_0,EEG_1,EEG_2,EEG_3,EEG_4,EEG_5,EEG_6,EEG_7,EEG_8,EEG_9,...,EEG_74990,EEG_74991,EEG_74992,EEG_74993,EEG_74994,EEG_74995,EEG_74996,EEG_74997,EEG_74998,EEG_74999
0,-41.473923,-45.176369,-48.871876,-52.430531,-55.698608,-58.338871,-59.919689,-60.365482,-59.915306,-58.619602,...,-39.994534,-40.737919,-41.243393,-41.563141,-41.477573,-40.687840,-39.171162,-37.272285,-35.384312,-33.797901
1,1.995628,1.976680,2.144622,2.307100,2.182211,1.700662,0.858995,-0.222909,-1.122924,-1.487486,...,4.151426,3.326329,2.465320,1.581539,0.663152,-0.336660,-1.501697,-2.669854,-3.677377,-4.576015
2,1.017115,1.657270,2.482397,3.356749,4.225514,5.210766,6.309640,7.299839,8.118546,8.818220,...,6.741571,5.831949,4.320967,2.681330,1.038642,-0.829033,-2.989935,-5.276758,-7.512856,-9.267806
3,11.683449,13.265381,15.650467,17.328840,16.866253,16.138889,15.232826,14.188766,12.432949,9.145377,...,-4.335334,-8.712267,-12.486134,-15.047755,-15.679938,-14.733905,-13.749776,-12.376304,-11.085991,-8.736834
4,17.023975,15.871809,14.530089,13.091481,11.459830,9.527679,7.319503,4.868694,2.319060,-0.036310,...,2.221342,1.686171,1.377594,1.298173,1.646075,2.646186,4.198915,5.949712,7.506319,8.671990


In [78]:
age = get_fft_mean_ampl(EEG,age)

coef = get_wavelet(EEG)
age = wav_features(age,coef)

age.head()

,ID,TARGET,device,mean_eeg,std_eeg,kurt_eeg,skew_eeg,max_eeg,min_eeg,length_list,...,wav_mean6,wav_std6,wav_min7,wav_max7,wav_mean7,wav_std7,wav_min8,wav_max8,wav_mean8,wav_std8
0,0,32,0,1.395848e-03,26.811350,129.538478,-2.978869,370.873413,-584.737732,971,...,1.520851e-02,4.628216,-35.175841,33.788319,-4.650481e-05,1.364214,-11.422063,4.929972,-2.596736e-04,0.156237
1,1,29,0,-6.471835e-04,11.881911,0.421025,-0.042748,58.905529,-66.786278,789,...,-4.621617e-02,6.917315,-6.526320,7.033817,-5.799839e-05,1.560359,-0.768676,0.833141,1.334174e-05,0.150531
2,2,36,0,1.041976e-02,97.740294,215.544923,3.324026,1944.065430,-2119.885986,295,...,1.440450e-02,8.770716,-24.799147,19.927536,1.321510e-04,1.605641,-8.060059,4.324459,-1.771710e-04,0.159557
3,3,56,0,-7.526106e-03,17.135283,145.579363,4.544951,440.595032,-190.482651,896,...,7.282955e-04,1.466351,-2.671790,3.093451,-1.654635e-05,0.279525,-5.199568,2.359752,-1.168486e-04,0.041260
4,4,60,1,-5.648294e-08,0.000034,1.346067,-0.322313,0.000124,-0.000185,939,...,4.602178e-08,0.000005,-0.000023,0.000023,-5.696861e-10,0.000002,-0.000024,0.000026,6.542273e-09,0.000001


In [79]:
age.columns

Index(['ID', 'TARGET', 'device', 'mean_eeg', 'std_eeg', 'kurt_eeg', 'skew_eeg',
       'max_eeg', 'min_eeg', 'length_list', 'occurencies_0', 'occurencies_1',
       'occurencies_2', 'occurencies_3', 'occurencies_4', 'record_quality',
       'SlowWaveSleep_percentage', 'breaks', 'fft_amp', 'wav_min0', 'wav_max0',
       'wav_mean0', 'wav_std0', 'wav_min1', 'wav_max1', 'wav_mean1',
       'wav_std1', 'wav_min2', 'wav_max2', 'wav_mean2', 'wav_std2', 'wav_min3',
       'wav_max3', 'wav_mean3', 'wav_std3', 'wav_min4', 'wav_max4',
       'wav_mean4', 'wav_std4', 'wav_min5', 'wav_max5', 'wav_mean5',
       'wav_std5', 'wav_min6', 'wav_max6', 'wav_mean6', 'wav_std6', 'wav_min7',
       'wav_max7', 'wav_mean7', 'wav_std7', 'wav_min8', 'wav_max8',
       'wav_mean8', 'wav_std8'],
      dtype='object')

In [80]:
age["record_quality"] = pd.Categorical(age["record_quality"] , ordered = False)
age["device"] = pd.Categorical(age["device"] , ordered = False)
dum = pd.get_dummies(age[["record_quality","device"]])

In [81]:
quantitative = age.ix[:,2:]

In [82]:
quantitative = quantitative.drop(["record_quality", "device"] , axis =1)

In [83]:
X_train = pd.concat([dum, quantitative], axis=1)

In [84]:
X_train.columns

Index(['record_quality_0', 'record_quality_1', 'device_0.0', 'device_1.0',
       'mean_eeg', 'std_eeg', 'kurt_eeg', 'skew_eeg', 'max_eeg', 'min_eeg',
       'length_list', 'occurencies_0', 'occurencies_1', 'occurencies_2',
       'occurencies_3', 'occurencies_4', 'SlowWaveSleep_percentage', 'breaks',
       'fft_amp', 'wav_min0', 'wav_max0', 'wav_mean0', 'wav_std0', 'wav_min1',
       'wav_max1', 'wav_mean1', 'wav_std1', 'wav_min2', 'wav_max2',
       'wav_mean2', 'wav_std2', 'wav_min3', 'wav_max3', 'wav_mean3',
       'wav_std3', 'wav_min4', 'wav_max4', 'wav_mean4', 'wav_std4', 'wav_min5',
       'wav_max5', 'wav_mean5', 'wav_std5', 'wav_min6', 'wav_max6',
       'wav_mean6', 'wav_std6', 'wav_min7', 'wav_max7', 'wav_mean7',
       'wav_std7', 'wav_min8', 'wav_max8', 'wav_mean8', 'wav_std8'],
      dtype='object')

In [85]:
Y_train = age["TARGET"]

# TEST

In [ ]:
dat_test= pd.read_csv(path + "test_input.csv" , header = 0 , sep =";")

In [110]:
age_test = pd.DataFrame()

In [111]:
EEG = get_eeg(dat_test)
age_test = get_device(age_test,dat_test)
age_test= get_eeg_features(EEG,age_test)

hypnogram = get_hypno(dat_test)
hyp = hypnogram.apply(preprocessing_hypno)

age_test = get_hypno_features(age_test,hyp)
age_test = get_fft_mean_ampl(EEG,age_test)

coef = get_wavelet(EEG)
age_test = wav_features(age_test,coef)

In [112]:
age_test.shape

(249, 53)

In [ ]:
age_test.head()

In [137]:
age_test.columns

Index(['device', 'mean_eeg', 'std_eeg', 'kurt_eeg', 'skew_eeg', 'max_eeg',
       'min_eeg', 'length_list', 'occurencies_0', 'occurencies_1',
       'occurencies_2', 'occurencies_3', 'occurencies_4', 'record_quality',
       'SlowWaveSleep_percentage', 'breaks', 'fft_amp', 'wav_min0', 'wav_max0',
       'wav_mean0', 'wav_std0', 'wav_min1', 'wav_max1', 'wav_mean1',
       'wav_std1', 'wav_min2', 'wav_max2', 'wav_mean2', 'wav_std2', 'wav_min3',
       'wav_max3', 'wav_mean3', 'wav_std3', 'wav_min4', 'wav_max4',
       'wav_mean4', 'wav_std4', 'wav_min5', 'wav_max5', 'wav_mean5',
       'wav_std5', 'wav_min6', 'wav_max6', 'wav_mean6', 'wav_std6', 'wav_min7',
       'wav_max7', 'wav_mean7', 'wav_std7', 'wav_min8', 'wav_max8',
       'wav_mean8', 'wav_std8'],
      dtype='object')

In [104]:
age_test.to_csv("age_test.csv")


In [113]:
age_test["record_quality"] = pd.Categorical(age_test["record_quality"] , ordered = False)
age_test["device"] = pd.Categorical(age_test["device"] , ordered = False)
dum = pd.get_dummies(age_test[["record_quality","device"]])

In [114]:
quantitative_test = age_test

In [115]:
quantitative_test = quantitative_test.drop(["record_quality", "device"] , axis =1)

In [116]:
X_test= pd.concat([dum, quantitative_test], axis=1)

In [118]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param=[{"max_features":list(range(6,len(X_train.columns),7)), "n_estimators" : list(range(50,350,50))}]
rf= GridSearchCV(RandomForestRegressor(),param,cv=5,n_jobs=-1)
rfOpt=rf.fit(X_train , Y_train)
# paramètres optimaux
print("Meilleur score = %f, nombre de features = %i, nombre d'arbres = %i" % (1. - rfOpt.best_score_,rfOpt.best_params_["max_features"], rfOpt.best_params_["n_estimators"]))

Meilleur score = 0.730303, nombre de features = 27, nombre d'arbres = 200


In [119]:
from sklearn.metrics import mean_squared_error
rf = RandomForestRegressor(max_features = 27 , n_estimators = 200)
rfFit = rf.fit(X_train, Y_train)
print("MSE=",mean_squared_error(Y_train,rfFit.predict(X_train)))

MSE= 13.6741394148


In [120]:
pred = np.round(rfFit.predict(X_test))

In [121]:
res = pd.DataFrame()
#res["ID"] = [i for i in range(581,830)]
res["TARGET"] = pred
res

,TARGET
0,34
1,39
2,42
3,39
4,38
5,38
6,57
7,34
8,41
9,49


In [122]:
res.to_csv("test_output2.csv")

In [149]:
rfFit.feature_importances_

array([  2.52983571e-03,   2.48700577e-03,   6.68196536e-06,
         2.38995805e-05,   1.36729860e-02,   2.28229890e-02,
         1.82548006e-02,   1.37949786e-02,   1.81616858e-02,
         1.20215981e-02,   1.91683966e-02,   4.08691984e-02,
         1.92257819e-02,   3.56217360e-02,   1.83026196e-02,
         2.27481069e-02,   5.95966198e-02,   2.00516985e-02,
         2.86430400e-02,   1.31547537e-02,   1.06369681e-02,
         1.63845010e-02,   1.13569481e-02,   1.83010700e-02,
         1.92273933e-02,   1.24355167e-02,   4.49803801e-02,
         1.64082070e-02,   4.74127475e-02,   1.38156938e-02,
         4.49427386e-02,   1.62000677e-02,   1.93936316e-02,
         1.93698582e-02,   2.13079855e-02,   1.57807562e-02,
         1.48597382e-02,   1.71283406e-02,   2.07738891e-02,
         9.41243353e-03,   1.27924390e-02,   1.38187733e-02,
         2.18206900e-02,   1.30758285e-02,   1.38995417e-02,
         1.51835535e-02,   1.96990042e-02,   1.10223023e-02,
         1.13786655e-02,